In [8]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.validation import make_valid
from tqdm import tqdm



In [15]:
# load flood data and reproject
#   set data input Path and flood map name
root_dir = Path ("C:/","Data","Floods","Germany", "raw_data", "SzenarioSelten") #specify the path to the folder holding the input data
print('Root: ',end='')
print(root_dir)
_flood_map_name = "DERP_RW_L.shp" #set the name for the flood map
flood_map=root_dir / _flood_map_name

#   set data output name and directory
#output_map=root_dir / "validated_geometries" / "DERP_RW_M_4326_subsample_valid_lmu.geojson"

print('File: ',end='')
print(flood_map)
assert flood_map.is_file()
gdf_to_edit=gpd.read_file(flood_map).to_crs(4326)



Root: C:\Data\Floods\Germany\raw_data\SzenarioSelten
File: C:\Data\Floods\Germany\raw_data\SzenarioSelten\DERP_RW_L.shp


In [16]:

# make geometries valid:
gdf_to_edit.geometry = gdf_to_edit.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

In [11]:
"""
Water depth class descriptions
T_class,Description
11,Water depth - classic flood area 0–0.5 m
12,Water depth - classic flood area > 0.5–1 m
13,Water depth - classic flood area > 1–2 m
14,Water depth - classic flood area > 2–4 m
15,Water depth - classic flood area > 4 m
16,Water depth - classic flood area > 0.5–2m (only in certain areas of Saxony)
17,Water depth - classic flood area > 2 m (only in certain areas of Saxony)
21,Water depth - indicative flood area 0–0.5 m
22,Water depth - indicative flood area > 0.5–1 m
23,Water depth - indicative flood area > 1–2 m
24,Water depth - indicative flood area > 2–4 m
25,Water depth - indicative flood area > 4 m
31,Water depth - flood-protected flood area 0–0.5 m
32,Water depth - flood-protected flood area > 0.5–1 m
33,Water depth - flood-protected flood area > 1–2 m
34,Water depth - flood-protected flood area > 2–4 m
35,Water depth - flood-protected flood area > 4 m
"""
#   define dictionary to relate water depth classes to water depths
#TODO discuss way to establish depth
w_depth_dict_lower={
    '1':0.0,
    '2':0.5,
    '3':1.0,
    '4':2.0,
    '5':4.0,
    '6':0.5, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}
w_depth_dict_mid={
    '1':0.25,
    '2':0.75,
    '3':1.5,
    '4':3.0,
    '5':4.0,
    '6':1.25, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}
w_depth_dict_upper={
    '1':0.5,
    '2':1.0,
    '3':2.0,
    '4':4.0,
    '5':4.0,
    '6':2.0, #only in areas of Saxony
    '7':2.0 #only in areas of Saxony
}

In [17]:
wdepth_class=[]
wdepth_l=[]
wdepth_m=[]
wdepth_u=[]
farea=[]

for i in range(len(gdf_to_edit)):
    farea.append(int(str(gdf_to_edit['T_class'][i])[0]))
    wdepth_class.append(int(str(gdf_to_edit['T_class'][i])[-1]))
    wdepth_l.append(float(w_depth_dict_lower[str(gdf_to_edit['T_class'][i])[-1]]))
    wdepth_m.append(float(w_depth_dict_mid[str(gdf_to_edit['T_class'][i])[-1]]))
    wdepth_u.append(float(w_depth_dict_upper[str(gdf_to_edit['T_class'][i])[-1]]))

gdf = gdf_to_edit.assign(w_depth_l=wdepth_l,w_depth_m=wdepth_m,w_depth_u=wdepth_u, depth_class=wdepth_class, flood_area=farea)


In [18]:
output_map = Path ("C:/","Data","Floods","Germany", "interim_data", "DERP_RW_L_4326_valid_lmu.geojson")
gdf.to_file(output_map)

In [7]:
import processing
# intersect basin polygons with flood map
processing.run("native:intersection", {'INPUT':'C:/Users/peregrin/OneDrive - Stichting Deltares/Documents/PhD Daniel Shared/Flood data/Ueberflutungstiefen_DERP/Flusshochwasser/SzenarioHaufig/DERP_RW_H.shp','OVERLAY':'C:/../../../../Data/Floods/basins/hybas_eu_lev01-12_v1c/hybas_eu_lev08_v1c.shp','INPUT_FIELDS':['T_class'],'OVERLAY_FIELDS':['HYBAS_ID','NEXT_DOWN','NEXT_SINK','MAIN_BAS','DIST_SINK','DIST_MAIN','SUB_AREA','UP_AREA','PFAF_ID','ORDER'],'OVERLAY_FIELDS_PREFIX':'','OUTPUT':'C:/Data/Floods/Germany/basin_intersections/DERP_RW_H_4326_hybas_intersections.geojson','GRID_SIZE':None})

ModuleNotFoundError: No module named 'processing'

In [3]:
#   set data input Path and flood map name 
root_dir = Path ("C:/","Data","Floods","Germany", "basin_intersections") #specify the path to the folder holding the input data

#C:\Data\Floods\Germany\fluvial_undefended\raw_subsample
print('Root: ',end='')
print(root_dir)
_flood_map_name = "DERP_RW_M_4326_hybas_intersections.geojson" #set the name for the flood map
flood_map=root_dir / _flood_map_name

#   set data output name and directory
output_map_dir=root_dir / _flood_map_name.split('.')[0]
#output_map_prot=root_dir / "DERP_RW_M_4326_wdepth_prot.shp"

print('File: ',end='')
print(output_map_dir)
# assert flood_map.is_file()

Root: C:\Data\Floods\Germany\basin_intersections
File: C:\Data\Floods\Germany\basin_intersections\DERP_RW_M_4326_hybas_intersections


In [4]:
flood_map_gdf=gpd.read_file(flood_map)
flood_map_gdf.head(2)


,T_class,w_depth,depth_clas,flood_area,HYBAS_ID,MAIN_BAS,PFAF_ID,ORDER,geometry
0,11,0.25,1,1,2080447840,2080023010,23267015,1,"MULTIPOLYGON (((8.16414 49.05517, 8.16415 49.0..."
1,11,0.25,1,1,2080447840,2080023010,23267015,1,"MULTIPOLYGON (((7.74782 49.16956, 7.74780 49.1..."


In [17]:
# hybas_ids=[]

# Iterate over unique hybas_ids
for unique_hybas_id in tqdm(flood_map_gdf['HYBAS_ID'].unique()):
    output_file=f'flood_basin_{unique_hybas_id}.geojson'
    output_path=output_map_dir / output_file
    # Select rows with the current unique value
    subset = flood_map_gdf[flood_map_gdf['HYBAS_ID'] == unique_hybas_id]  # Assuming there are at least 5 rows for each unique value
    # Export the subset as GeoJSON
    subset.to_file(output_path, driver='GeoJSON')



  0%|          | 0/61 [00:00<?, ?it/s]

100%|██████████| 61/61 [15:46<00:00, 15.51s/it] 


In [11]:
subset

,T_class,w_depth,depth_clas,flood_area,HYBAS_ID,MAIN_BAS,PFAF_ID,ORDER,geometry
829,15,4.0,5,1,2080429670,2080023010,23264010,2,"MULTIPOLYGON (((8.29181 49.99260, 8.29182 49.9..."
